In [1]:
%serialconnect

Connecting to Serial /dev/ttyUSB1 baud=115200 
Ready.



In [3]:
from i2cmodule import i2c
print(0x77, (i2c.scan()))

119 [64, 119]


In [5]:
import time
i2c.writeto(0x40, b'\xFE')  # resets chip
time.sleep(0.2)


In [87]:
%%writefile si7021_funcs.py

from i2cmodule import i2c
import ustruct, time, math

def setupSI7021():
    i2c.writeto(0x40, b'\xFE')  # resets chip
    time.sleep_ms(20)  # got to delay or you get an error if call next function too soon
    
def SI7021checkchip():
    i2c.writeto(0x40, b'\xFA\x0F')
    sna = i2c.readfrom(0x40, 8)
    i2c.writeto(0x40, b'\xFC\xC9')
    snb = i2c.readfrom(0x40, 6)
    i2c.writeto(0x40, b'\x84\xB8')
    firmr = i2c.readfrom(0x40, 1)
    print("SNA %s %s %s %s  SNB %s %s %s %s  firmware %s" % (hex(sna[0]), hex(sna[2]), hex(sna[4]), hex(sna[6]), hex(snb[0]), hex(snb[1]), hex(snb[3]), hex(snb[4]), hex(firmr[0])))
    return (snb[0] == 21) # identifies the Si7021 type chip

def SI7021printstatus():
    reg1 = i2c.readfrom_mem(0x40, 0xE7, 1)[0]
    heater = i2c.readfrom_mem(0x40, 0x11, 1)[0]
    print("MeasRes:%s VDD:%s heater-on:%s heater:%s" % (hex(reg1 & 0x81), hex(reg1 & 0x40), hex(reg1 & 0x04), hex(heater & 0x0F)))

def SI7021setheater(hheater):
    # hheater to be between 0 and 15
    reg1 = i2c.readfrom_mem(0x40, 0xE7, 1)[0]
    nreg1 = (reg1 & 0xFB) if (hheater == 0) else (reg1 | 0x04) 
    i2c.writeto_mem(0x40, 0xE6, bytes([nreg1]))
    
    heater = i2c.readfrom_mem(0x40, 0x11, 1)[0]
    nheater = ((heater & 0xF0) | hheater); 
    i2c.writeto_mem(0x40, 0x51, bytes([nheater]))

def SI7021humiditytemp():
        # E5 is hold master mode, didn't used to work, but now does while no hold F5 stopped working
        # (the clock stretching doesn't seem to apply anyway)
    i2c.writeto(0x40, b'\xE5')  
    time.sleep_ms(20)   # give it time to take a reading or it fails
    bh = i2c.readfrom(0x40, 2)
    rh = ustruct.unpack(">H", bh)[0] & 0xFFFC
    bt = i2c.readfrom_mem(0x40, 0xE0, 2)
    rt = ustruct.unpack(">H", bt)[0] & 0xFFFC
    return ((125.0*rh)/65536)-6, ((175.25*rt)/65536)-46.85 

async def aSI7021humiditytemp(uasyncio):
    i2c.writeto(0x40, b'\xE5')  
    await uasyncio.sleep_ms(20)   # give it time to take a reading or it fails
    bh = i2c.readfrom(0x40, 2)
    rh = ustruct.unpack(">H", bh)[0] & 0xFFFC
    bt = i2c.readfrom_mem(0x40, 0xE0, 2)
    rt = ustruct.unpack(">H", bt)[0] & 0xFFFC
    return ((125.0*rh)/65536)-6, ((175.25*rt)/65536)-46.85 

def DewpointTemperature(humid, temp):
    A, B, C = 8.1332, 1762.39, 235.66
    svp = 10**(A - B/(temp + C))*133.322387415
    pvp = svp*humid/100
    return -C - B/(math.log10(pvp/133.322387415) - A)


Writing si7021_funcs.py



In [88]:
%sendtofile --source si7021_funcs.py

Sent 57 lines (2333 bytes) to si7021_funcs.py.


In [84]:
i2c.writeto(0x40, b'\xFA\x0F')
sna = i2c.readfrom(0x40, 8)
i2c.writeto(0x40, b'\xFC\xC9')
snb = i2c.readfrom(0x40, 6)
i2c.writeto(0x40, b'\x84\xB8')
firmr = i2c.readfrom(0x40, 1)
print("SNA %s %s %s %s  SNB %s %s %s %s  firmware %s" % (hex(sna[0]), hex(sna[2]), hex(sna[4]), hex(sna[6]), hex(snb[0]), hex(snb[1]), hex(snb[3]), hex(snb[4]), hex(firmr[0])))


SNA 0x0 0x39 0x64 0x3c  SNB 0x15 0xff 0xff 0xff  firmware 0x20


In [79]:
from si7021_funcs import setupSI7021, SI7021humiditytemp, DewpointTemperature
from si7021_funcs import SI7021checkchip, SI7021printstatus, SI7021setheater


In [86]:
setupSI7021()
time.sleep_ms(20)
SI7021checkchip()
SI7021printstatus()

SNA 0x0 0x39 0x64 0x3c  SNB 0x15 0xff 0xff 0xff  firmware 0x20
MeasRes:0x0 VDD:0x0 heater-on:0x0 heater:0x0


In [75]:
print(SI7021humiditytemp())

(69.37842, 20.16302)


In [26]:
i2c.writeto(0x40, b'\xF5')  # clock stretching hold type E5 seems not to work in micropython
time.sleep_ms(120)   # give it time to take a reading or it fails
bh = i2c.readfrom(0x40, 2)


Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
OSError: [Errno 19] ENODEV


In [19]:
i2c.writeto(0x40, b'\xFE')  # resets chip


In [71]:
import ustruct

t0 = time.ticks_ms()
i2c.writeto(0x40, b'\xE5')
time.sleep_ms(30)
bh = i2c.readfrom(0x40, 2)
bt = i2c.readfrom_mem(0x40, 0xE0, 2)
print(time.ticks_ms() - t0)
rt = ustruct.unpack(">H", bt)[0] & 0xFFFC
t = ((175.25*rt)/65536)-46.85 
print(t)


33
20.12023


In [38]:
print(bh)

b'b$'
